# Step 4: Add a model building CI/CD pipeline

In this step we create an automated CI/CD pipeline for model building using [Amazon SageMaker Projects](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects.html). 

![](img/six-steps-4.png)

We use a [SageMaker-provided MLOps project template for model building and training](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-sm.html#sagemaker-projects-templates-code-commit) to provision a ready-to use CI/CD workflow automation with [AWS CodePipeline](https://aws.amazon.com/codepipeline/) and an [AWS CodeCommit](https://aws.amazon.com/codecommit/) code repository.

SageMaker project templates offer you the following choice of code repositories, workflow automation tools, and pipeline stages:
- **Code repository**: AWS CodeCommit or third-party Git repositories such as GitHub and Bitbucket
- **CI/CD workflow automation**: AWS CodePipeline or Jenkins
- **Pipeline stages**: Model building and training, model deployment, or both

In [2]:
import boto3
import sagemaker 

In [3]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

Stored variables and their in-db values:
abalone_dataset_file_name                -> 'abalone.csv'
abalone_dataset_local_url                -> '../dataset/abalone.csv'
auto_ml_job_name                         -> 'automl-asinproc-24-14-33-56'
bucket_name                              -> 'sagemaker-us-east-1-906545278380'
bucket_prefix                            -> 'from-idea-to-prod/xgboost'
customers_count                          -> 10000
customers_feature_group_name             -> 'fscw-customers-07-20-17-46'
data_bucket                              -> 'sagemaker-us-east-1-906545278380'
data_uploaded                            -> True
domain_id                                -> 'd-r8pbvl3oamh6'
dw_flow_file_url                         -> 's3://sagemaker-us-east-1-906545278380/feature-sto
dw_output_name                           -> '928854ec-259e-4130-8e0f-b65221b27d6e.default'
endpoint_name                            -> 'reorder-classifier-2022-07-20-19-45-09-338'
execution_role      

## Create an MLOps project
⭐ You can create a project programmatically in this notebook (Option 1) or in Studio (Option 2). Option 1 is recommended as it requires no manual input. Option 2 is given to demonstrate [**Create Project** UX flow](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-create.html).

### Option 1: Create project programmatically
We use `boto3` to create an MLOps project via a SageMaker API.

### Option 2: Create a project in Studio
<div class="alert alert-info"> 💡 <strong> Skip this section if you created a project programmatically </strong>

To create a project in Studio:

1. In the Studio sidebar, choose the **SageMaker resources** icon.
2. Select **Projects** from the dropdown list.
3. Choose **Create project**.
    - The **Create project** tab opens displaying a list of available templates.
4. For **SageMaker project templates**, choose **SageMaker templates**. 
5. Choose **MLOps template for model building and training**
6. Choose **Select project template**.

![](img/create-mlops-project.png)

The **Create project** tab changes to display **Project details**.

![](img/project-details.png)

Enter the following information:
- For **Project details**, enter a name and description for your project. Note the name requirements.
- Optionally, add tags, which are key-value pairs that you can use to track your projects.

Choose **Create project** and wait for the project to appear in the Projects list.

### Resolve issues with project creation
❗ If you see an error message similar to:
```
Your project couldn't be created
Studio encountered an error when creating your project. Try recreating the project again.

CodeBuild is not authorized to perform: sts:AssumeRole on arn:aws:iam::XXXX:role/service-role/AmazonSageMakerServiceCatalogProductsCodeBuildRole (Service: AWSCodeBuild; Status Code: 400; Error Code: InvalidInputException; Request ID: 4cf59a54-0c59-476a-a970-0ac656db4402; Proxy: null)
```

see steps 5-6 of [SageMaker Studio Permissions Required to Use Projects](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-studio-updates.html). Make sure you have all required project roles listed in the **Apps** card under **Projects**. 

💡 If you don't have these roles, you must follow [Shut Down and Update SageMaker Studio and Studio Apps](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-tasks-update.html) instructions to update the domain. You must shutdown both JupyterServer and KernelGateway apps. After you shutdown all apps, go to Amazon SageMaker **Control Panel**, choose **Configure app** on the **App** card. Click through all **Next** in configuration panes and choose **Submit**. This will update the domain and create all needed project roles automatically.

## Configure the MLOps project
The project takes about 3-5 min to be created. The project runs a provided default model building pipeline automatically as soon as it has been created.
The project templates deploys the following architecture in your environment:

![](img/mlops-model-build-train.png)

The main components are:
1. The project template is made available through SageMaker Projects and AWS Service Catalog portfolio
2. A CodePipeline pipeline with two stages - `Source` to download the source code from a CodeCommit repository and `Build` to create and execute a SageMaker pipeline
3. A default SageMaker pipeline with model build, train, and register workflow
4. A seed code repository in CodeCommit with a provided default version of the scaffolding code

This project contains all the required code and the insfrastructure to implement an automated CI/CD pipeline. 
To start using the project with your pipeline, you need to complete the following steps:
1. Clone the provided project seed code to Studio environment
2. Replace the pipeline creation sample code with your pipeline construction code, as implemented in the step 3 notebook
3. Modify the `codebuild-buildspec.yml` file to reference the correct Python module name and to set project parameters

Next sections guide you through these steps. For detailed instructions and a hands-on example, refer to the development guide [SageMaker MLOps Project Walkthrough](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough.html).

### 1. Clone the project seed code to the Studio file system
1. Choose **SageMaker resources** in the Studio sidebar and then select **Projects**
2. Select the name of the project you created and double-click on it to open the project details tab
3. In the project tab, choose **Repositories**, and in the **Local path** column for the repository choose **clone repo....**
4. In the dialog box that appears choose **Clone Repository**

![](img/clone-project-repo.png)

When clone of the repository is complete, the local path appears in the **Local path** column. Choose the path to open the local folder that contains the repository code in Studio.

### 2. Replace pipeline construction code
In Studio file browser navigate to the `pipelines` folder inside the project folder and rename the `abalone` folder to `fromideatoprod`.

Copy the `preprocessing.py` file that we created in the step 2 from the `amazon-sagemaker-from-idea-to-production` folder to the `pipelines/fromideatoprod` folder in the project's code repository folder, which looks like `sagemaker-<project-name>-<project-id>-modelbuild`.

Execute the following cell to write pipeline construction code to the file `pipeline.py`. We re-use the code from the step 3 notebook as the function `get_pipeline()`.

In [5]:
%%writefile pipeline.py

import os

import boto3
import sagemaker
import sagemaker.session

from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import (
    ConditionStep,
)
from sagemaker.workflow.functions import (
    JsonGet,
)
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import (
    ProcessingStep,
    TrainingStep,
)
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.model_step import ModelStep
from sagemaker.model import Model
from sagemaker.workflow.pipeline_context import PipelineSession


BASE_DIR = os.path.dirname(os.path.realpath(__file__))

def get_sagemaker_client(region):
     """Gets the sagemaker client.

        Args:
            region: the aws region to start the session
            default_bucket: the bucket to use for storing the artifacts

        Returns:
            `sagemaker.session.Session instance
        """
     boto_session = boto3.Session(region_name=region)
     sagemaker_client = boto_session.client("sagemaker")
     return sagemaker_client


def get_session(region, default_bucket):
    """Gets the sagemaker session based on the region.

    Args:
        region: the aws region to start the session
        default_bucket: the bucket to use for storing the artifacts

    Returns:
        `sagemaker.session.Session instance
    """

    boto_session = boto3.Session(region_name=region)

    sagemaker_client = boto_session.client("sagemaker")
    runtime_client = boto_session.client("sagemaker-runtime")
    return sagemaker.session.Session(
        boto_session=boto_session,
        sagemaker_client=sagemaker_client,
        sagemaker_runtime_client=runtime_client,
        default_bucket=default_bucket,
    )

def get_pipeline_session(region, default_bucket):
    """Gets the pipeline session based on the region.

    Args:
        region: the aws region to start the session
        default_bucket: the bucket to use for storing the artifacts

    Returns:
        PipelineSession instance
    """

    boto_session = boto3.Session(region_name=region)
    sagemaker_client = boto_session.client("sagemaker")

    return PipelineSession(
        boto_session=boto_session,
        sagemaker_client=sagemaker_client,
        default_bucket=default_bucket,
    )

def get_pipeline_custom_tags(new_tags, region, sagemaker_project_arn=None):
    try:
        sm_client = get_sagemaker_client(region)
        response = sm_client.list_tags(
            ResourceArn=sagemaker_project_arn)
        project_tags = response["Tags"]
        for project_tag in project_tags:
            new_tags.append(project_tag)
    except Exception as e:
        print(f"Error getting project tags: {e}")
    return new_tags


def get_pipeline(
    region,
    sagemaker_project_arn=None,
    role=None,
    default_bucket=None,
    input_data_url=None,
    bucket_prefix="from-idea-to-prod/xgboost",
    model_package_group_name="from-idea-to-prod-model-group",
    pipeline_name="from-idea-to-prod-pipeline",
    base_job_prefix="from-idea-to-prod-pipeline",
    processing_instance_type="ml.c5.xlarge",
    training_instance_type="ml.m5.xlarge",
):
    """Gets a SageMaker ML Pipeline instance.

    Args:
        region: AWS region to create and run the pipeline.
        role: IAM role to create and run steps and pipeline.
        default_bucket: the bucket to use for storing the artifacts

    Returns:
        an instance of a pipeline
    """
    sagemaker_session = get_session(region, default_bucket)
    if role is None:
        role = sagemaker.session.get_execution_role(sagemaker_session)

    pipeline_session = get_pipeline_session(region, default_bucket)

    # Set S3 urls for processed data
    train_s3_url = f"s3://{default_bucket}/{bucket_prefix}/train"
    validation_s3_url = f"s3://{default_bucket}/{bucket_prefix}/validation"
    test_s3_url = f"s3://{default_bucket}/{bucket_prefix}/test"
    
    # Set S3 url for model artifact
    output_s3_url = f"s3://{default_bucket}/{bucket_prefix}/output"

    # Parameters for pipeline execution
    # Set processing instance type
    process_instance_type_param = ParameterString(
        name="ProcessingInstanceType",
        default_value=processing_instance_type,
    )

    # Set training instance type
    train_instance_type_param = ParameterString(
        name="TrainingInstanceType",
        default_value=training_instance_type,
    )

    # Set training instance count
    train_instance_count_param = ParameterInteger(
        name="TrainingInstanceCount",
        default_value=1
    )

    # Set model approval param
    model_approval_status_param = ParameterString(
        name="ModelApprovalStatus", default_value="PendingManualApproval"
    )

    # Set S3 url for input dataset
    input_s3_url_param = ParameterString(
        name="InputDataUrl",
        default_value=input_data_url,
    )

    # processing step for feature engineering
    sklearn_processor = SKLearnProcessor(
            framework_version="0.23-1",
            role=role,
            instance_type=process_instance_type_param,
            instance_count=1,
            sagemaker_session=pipeline_session,
            base_job_name=f"{base_job_prefix}/preprocess",
        )

    # Define processing step
    step_process = ProcessingStep(
        name="PreprocessData",
        processor=sklearn_processor,
        inputs=[
            ProcessingInput(source=input_s3_url_param, destination="/opt/ml/processing/input")
        ],
        outputs=[
            ProcessingOutput(output_name="train_data", source="/opt/ml/processing/output/train", 
                             destination=train_s3_url),
            ProcessingOutput(output_name="validation_data", source="/opt/ml/processing/output/validation",
                             destination=validation_s3_url),
            ProcessingOutput(output_name="test_data", source="/opt/ml/processing/output/test",
                             destination=test_s3_url),
        ],
        code=os.path.join(BASE_DIR, "preprocessing.py"),
    #    job_arguments=["--arg1", "arg1_value", "--arg2", "arg2_value"],
    )

    # Training step for generating model artifacts
    training_image = sagemaker.image_uris.retrieve("xgboost", region=region, version="latest")

    # Instantiate an XGBoost estimator object
    estimator = sagemaker.estimator.Estimator(
        image_uri=training_image,
        role=role, 
        instance_type=train_instance_type_param,
        instance_count=train_instance_count_param,
        output_path=output_s3_url,
        sagemaker_session=pipeline_session,
        base_job_name=f"{base_job_prefix}/train",
    )

    # Define its hyperparameters
    estimator.set_hyperparameters(
        num_round=150, # the number of rounds to run the training
        max_depth=5, # maximum depth of a tree
        eta=0.5, # step size shrinkage used in updates to prevent overfitting
        alpha=2.5, # L1 regularization term on weights
        objective="binary:logistic",
        eval_metric="auc", # evaluation metrics for validation data
        subsample=0.8, # subsample ratio of the training instance
        colsample_bytree=0.8, # subsample ratio of columns when constructing each tree
        min_child_weight=3, # minimum sum of instance weight (hessian) needed in a child
        early_stopping_rounds=10, # the model trains until the validation score stops improving
        verbosity=1, # verbosity of printing messages
    )

    # Define training step
    step_train = TrainingStep(
        name="Train",
        estimator=estimator,
        inputs={
            "train": TrainingInput(
                s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                    "train_data"
                ].S3Output.S3Uri,
                content_type="text/csv",
            ),
            "validation": TrainingInput(
                s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                    "validation_data"
                ].S3Output.S3Uri,
                content_type="text/csv",
            ),
        },
    )

    # Define register step
    step_register = RegisterModel(
        name="RegisterModel",
        estimator=estimator,
        model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
        content_types=["text/csv"],
        response_types=["text/csv"],
        inference_instances=["ml.t2.medium", "ml.m5.large"],
        transform_instances=["ml.m5.large"],
        model_package_group_name=model_package_group_name,
        approval_status=model_approval_status_param,
    )

    # Pipeline instance
    pipeline = Pipeline(
        name=pipeline_name,
        parameters=[
            process_instance_type_param,
            train_instance_type_param,
            train_instance_count_param,
            model_approval_status_param,
            input_s3_url_param,
        ],
        steps=[step_process, step_train, step_register],
        sagemaker_session=pipeline_session,
    )
    
    return pipeline

Overwriting pipeline.py


Copy this `pipeline.py` file from the `amazon-sagemaker-from-idea-to-production` folder to the `pipelines/fromideatoprod` folder in the project's code repository folder (replace the existing `pipeline.py` file in the folder).

### 3. Modify the build specification file
You must modify the `codebuild-buildspec.yml` file in the project folder to reflect the new name of Python module with your pipeline and set project-specific parameters.

First, print the value of `input_s3_url` variable with the S3 path to the source dataset. You must pass this value to the pipeline:

In [10]:
input_s3_url

's3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/input/bank-additional-full.csv'

Second, replace the value of the `input_data_url` parameter in the following cell with the value of `input_s3_url`. Execute the cell to create a build spec file.

In [6]:
%%writefile codebuild-buildspec.yml

version: 0.2

phases:
  install:
    runtime-versions:
      python: 3.8
    commands:
      - pip install --upgrade --force-reinstall . "awscli>1.20.30"
  
  build:
    commands:
      - export PYTHONUNBUFFERED=TRUE
      - export SAGEMAKER_PROJECT_NAME_ID="${SAGEMAKER_PROJECT_NAME}-${SAGEMAKER_PROJECT_ID}"
      - |
        run-pipeline --module-name pipelines.fromideatoprod.pipeline \
          --role-arn $SAGEMAKER_PIPELINE_ROLE_ARN \
          --tags "[{\"Key\":\"sagemaker:project-name\", \"Value\":\"${SAGEMAKER_PROJECT_NAME}\"}, {\"Key\":\"sagemaker:project-id\", \"Value\":\"${SAGEMAKER_PROJECT_ID}\"}]" \
          --kwargs "{\"region\":\"${AWS_REGION}\",\"sagemaker_project_arn\":\"${SAGEMAKER_PROJECT_ARN}\",\"role\":\"${SAGEMAKER_PIPELINE_ROLE_ARN}\",\"default_bucket\":\"${ARTIFACT_BUCKET}\",\"input_data_url\":\"s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/input/bank-additional-full.csv\"}"
      - echo "Create/Update of the SageMaker Pipeline and execution completed."

Overwriting codebuild-buildspec.yml


Copy the `codebuild-buildspec.yml` file from the `amazon-sagemaker-from-idea-to-production` folder to the project's code repository folder. Replace the the existing build spec file.

We did three changes in the build spec file:
1. Modified the `run-pipeline` `--module-name` parameter value from `pipelines.abalone.pipeline` to the new path `pipelines.fromideatoprod.pipeline`
2. Removed some parameters from the `kwargs` list to make use of `get_pipeline()` function default parameter values
3. Added an Amazon S3 url to the source data to the `kwargs` parameter list

Finally, open the `setup.py` file in the project's code repository folder and replace the line `required_packages = ["sagemaker==2.XX.0"]` with `required_packages = ["sagemaker"]`. Save your changes.

Now you are ready to launch the CI/CD model building pipeline.

## Run CI/CD model building pipeline
❗ Make sure you are in the local folder that contains the **repository code** in Studio file browser. The folder name looks like `sagemaker-<project-name>-<project-id>-modelbuild`.

Choose the Git icon in the Studio sidebar. Stage, commit, and push all the changes in the project repository. 

Alternatively, you can run all git commands from the Studio system terminal:
```sh
cd ~/<PROJECT-FOLDER>/<PROJECT-CODE-REPOSITORY-FOLDER>
git add -A
git commit -am "customize project"
git push
```

After pushing your code changes, the MLOps project initiates a run of the CodePipeline pipeline that updates and executes the SageMaker model building pipeline. This new pipeline execution creates a new model version in the model package group in the SageMaker model registry.

You can follow up the execution of the pipeline in **SageMaker resources** > **Pipelines**.

## View the details of a new model version
1. In the Studio sidebar, choose the **SageMaker resources** icon
2. Select **Model registry** from the dropdown list
3. Select the name of the model package group you created in the step 3 notebook (`from-idea-to-prod-model-group`) and double-click on it to open the model group
4. In the list of model versions, double-click on the latest version of the model

![](img/model-package-group.png)

On the model version tab that opens, you can browse activity, [model version details](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-details.html), and [data lineage](https://docs.aws.amazon.com/sagemaker/latest/dg/lineage-tracking.html). 

![](img/model-version-details.png)

In a real-world project you add various model attributes and additional model version metadata such as [model quality metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html), [explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-explainability.html) and [bias](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html) reports, load test data, and [inference recommender](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-recommender.html).

## Continue with the step 5
open the step 5 [notebook](05-deploy.ipynb).

## Further development ideas
- You can use a SageMaker-provided [MLOps template for model building, training, and deployment with third-party Git repositories using Jenkins](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-sm.html#sagemaker-projects-templates-git-jenkins)
- Create a [custom SageMaker project template](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-custom.html) to cover your specific project requirements

## Additional resources
- [Amazon SageMaker Pipelines lab in SageMaker Immersion Day](https://catalog.us-east-1.prod.workshops.aws/workshops/63069e26-921c-4ce1-9cc7-dd882ff62575/en-US/lab6)
- [Enhance your machine learning development by using a modular architecture with Amazon SageMaker projects](https://aws.amazon.com/blogs/machine-learning/enhance-your-machine-learning-development-by-using-a-modular-architecture-with-amazon-sagemaker-projects/)
- [Dive deep into automating MLOps](https://www.youtube.com/watch?v=3_cHnk9VSfQ)
- [SageMaker MLOps Project Walkthrough](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough.html)

# Shutdown kernel

In [10]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>